In [619]:
import numpy as np

In [620]:
heightCol = 22
heightPipeMin = 2
heightPipeMax = 16
heightGapMin = 2
heightBoundaryTotal = 2
lengthLevel = 25
lengthEmptySpace = 5
nLevels = 50
pathLevels = "./levels/"


In [621]:
tiles =  {
    "X" : ["solid","ground"],
    "S" : ["solid","breakable"],
    "-" : ["passable","empty"],
    "?" : ["solid","question block", "full question block"],
    "Q" : ["solid","question block", "empty question block"],
    "E" : ["enemy","damaging","hazard","moving"],
    "<" : ["solid","top-left pipe","pipe"],
    ">" : ["solid","top-right pipe","pipe"],
    "[" : ["solid","left pipe","pipe"],
    "]" : ["solid","right pipe","pipe"],
    "o" : ["coin","collectable","passable"]
}

In [622]:
def getEmptyTilesArray(length):
    return getTilesArray("-", length)

def getTilesArray(tile, length):
    return list(np.repeat(tile, length))

def appendGroundAndSky(li):
    return np.append(np.append("X", li),"X")

def getSolidTile(length):
    solid = ["X"]
    tile = []
    for i in range(length):
        tile = np.append(tile, solid)
    return getCol(tile, 0)

def getCol(li, dim=1):
    return np.expand_dims(li, dim)

In [623]:
def getEmptyCol(length=heightCol-2):
    emptyCol = getEmptyTilesArray(length)
    emptyCol = getCol(emptyCol)
    return emptyCol

def getEmptyCols(length):
    emptyCol = getEmptyCol(length)
    return np.append(emptyCol, emptyCol, 1)

def getEmptyColsBounded():
    empty = getEmptyCols(heightCol-2)
    solid = getSolidTile(2)
    return np.concatenate((solid, empty, solid))


In [624]:
def getPipeBody(length, side):
    pipeBody = ""
    if side == "left":
        pipeBody = "["
    elif side == "right":
        pipeBody = "]"
    return getTilesArray(pipeBody, length)

def getPipeHead(side):
    if side == "left":
        return "<"
    elif side == "right":
        return ">"
    return ""

def getPipeSide(side, direction="ascending", length="4"):
    if direction == "ascending":
        return getCol(np.append(getPipeHead(side), getPipeBody(length, side)))
    elif direction == "descending":
        return getCol(np.append(getPipeBody(length, side), getPipeHead(side)))
    return []
    
def getPipe(direction="ascending", length=5):
    pipeLeft = getPipeSide(side="left", direction=direction, length=length - 1)
    pipeRight = getPipeSide(side="right", direction=direction, length=length - 1)
    return np.append(pipeLeft, pipeRight, 1)

def getPipeCols():
    [pipeHeight, gapHeight] = getRandPipeAndGapHeight()
    pipeBottom = getPipe("ascending", pipeHeight)
    pipeGap = getEmptyCols(gapHeight)
    heightPipeTop = heightCol - gapHeight - pipeHeight - heightBoundaryTotal
    pipeTop = getPipe("descending", heightPipeTop)
    boundary = getSolidTile(2)
    return np.concatenate((boundary, pipeTop, pipeGap, pipeBottom, boundary), axis=0)

def getRandPipeAndGapHeight():
    heightPipe = np.random.randint(heightPipeMin,(heightPipeMax + 1))
    boundGapHigh = heightCol - heightPipe - heightPipeMin - heightPipeMin
    heightGap = np.random.randint(heightGapMin, (boundGapHigh + 1))
    return (heightPipe, heightGap)




In [625]:
def getLevelBlock():
    pipe = getPipeCols()
    empty = getEmptyColsBounded()
    return np.concatenate((empty, empty, empty, pipe), 1)

def getLevelBlocks(length):
    emptyBlock = getEmptyColsBounded()
    levelBlock = emptyBlock
    # add empty space for the beginning of the level
    for _ in range(lengthEmptySpace - 1):
        levelBlock = np.concatenate((levelBlock, emptyBlock), 1)
    for _ in range(length):
        levelBlock = np.concatenate((levelBlock, getLevelBlock()), 1)
    # add empty space for the ending of the level
    for _ in range(lengthEmptySpace):
        levelBlock = np.concatenate((levelBlock, emptyBlock), 1)
    return levelBlock

def getLevel(length = lengthLevel):
    return getLevelBlocks(length)

In [626]:
level = getLevel()
print(level)
np.savetxt("level.txt", level, fmt="%s")

[['X' 'X' 'X' ... 'X' 'X' 'X']
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ...
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ['X' 'X' 'X' ... 'X' 'X' 'X']]


In [627]:

for nLevel in range(1, nLevels + 1):
    level = getLevel()
    fileName = pathLevels + "level_%03d.txt" % nLevel
    np.savetxt(fileName, level, fmt="%s")
